In [17]:
import sys, os
import pickle
import random
import multiprocess
from multiprocess import Pool

import numpy as np

from multistrand.objects import *
from multistrand.options import Options, Literals
from multistrand.system import SimSystem, calculate_energy
from multistrand.utils.thermo import C2K

In [52]:
def print_trajectory(o):
    seqstring=''
    for i in range(len(o.full_trajectory)): # go through each output microstate of the trajectory
        time = o.full_trajectory_times[i]   # time at which this microstate is entered
        states = o.full_trajectory[i]       # this is a list of the complexes present in this tube microstate
        newseqs = []
        for state in states: newseqs += [ state[3] ]   # extract the strand sequences in each complex (joined by "+" for multistranded complexes)
        newseqstring = ' '.join(newseqs)    # make a space-separated string of complexes, to represent the whole tube system sequence
        if not newseqstring == seqstring :
            print(newseqstring)
            seqstring=newseqstring          # because strand order can change upon association of dissociation, print it when it changes
        structs = []
        for state in states: structs += [ state[4] ]   # similarly extract the secondary structures for each complex
        tubestruct = ' '.join(structs)      # give the dot-paren secondary structure for the whole test tube
        dG=0
        for state in states: dG += state[5]
        print('%s t=%11.9f seconds, dG=%6.2f kcal/mol' % (tubestruct,time, dG))

        # Needlessly verify that the reported trajectory energies are the Tube_Energy values
        dGv=0
        for state in states:
            cs=state[3].split('+')
            st=state[4]
            dGv += calculate_energy( [Complex( strands=[Strand(sequence=s) for s in cs], structure=st)], o, EnergyType.tube)[0]
        if not dGv == dG: print("Energy Mismatch")

In [53]:
toehold_length = 7
toehold_seq = 'ATGTGGAGGG'
bm_design = "GGTGAGTTTGAGGTTGA"
incum_extra_design = "TGGTGTTTGTGGGTGT"

# creating Domain objects
toehold = Domain(name="toehold", sequence=toehold_seq[0:toehold_length])
branch_migration = Domain(name="bm", sequence=bm_design)

# invader
invader = branch_migration + toehold
invader.name = "invader"

# target (substrate)
toehold_extra = Domain(name="toehold_extra", sequence=toehold_seq[toehold_length:]) 
target = toehold_extra.C + toehold.C + branch_migration.C
target.name = "target"

# incumbent
incumbent_extra = Domain(name="incumbent_extra", sequence=incum_extra_design)
incumbent = incumbent_extra+branch_migration
incumbent.name = "incumbent"

In [54]:
# creates the target-incumbent start complex  
start_complex_substrate_incumbent = Complex(strands=[target, incumbent], structure="..(+.)")
# creates invader complex. 
start_complex_incoming = Complex(strands=[invader], structure="..") 

# creates a complex for a "succcessful displacement" stop condition. This is the incumbent strand forming a complex of its own which means it has been displaced.
complete_complex_success = Complex(strands=[incumbent], structure="..")
success_stop_condition = StopCondition("SUCCESS", [(complete_complex_success, Literals.dissoc_macrostate, 0)])

# complex to create failed displacement stop condition; incumbent falls off.   
failed_complex = Complex(strands=[invader], structure="..")  
failed_stop_condition = StopCondition("FAILURE", [(failed_complex, Literals.dissoc_macrostate, 0)]) 
    

In [56]:
o = Options(
            simulation_mode="First Step", 
            num_simulations=1, 
            simulation_time=10.0,  # note the 10 second simulation time, to make sure simulations finish
            dangles="Some", 
            temperature=25 + C2K, 
            verbosity=2)

o.DNA23Metropolis()
o.start_state = [start_complex_incoming, start_complex_substrate_incumbent]
o.stop_conditions = [success_stop_condition, failed_stop_condition]

In [57]:
s = SimSystem(o)
s.start()

-8721383166943299272: [2] '4:invader': 0.0 
GGTGAGTTTGAGGTTGAATGTGGA
........................

-8721383166943299272: [1] '6:target,7:incumbent': -24.873952926003547 
CCCTCCACATTCAACCTCAAACTCACC+TGGTGTTTGTGGGTGTGGTGAGTTTGAGGTTGA
..........(((((((((((((((((+................)))))))))))))))))



In [61]:
s.

In [62]:
o

In [51]:
o.full_trajectory


[]

In [65]:
def load_file(dirname, filename):
    fullname = os.path.join(dirname, filename)

    f = open(fullname, 'rb')
    res = pickle.load(f)
    f.close()

    return res

def process_dataset(dataset):
    # essentially identifies trajectories of each kind and takes the first passage times
    
    forward = [i for i in dataset if i.tag == "SUCCESS"]
    forward_array = np.zeros(len(forward))
    forward_array[:] = [i.time for i in forward]

    # if after 10 simulated seconds, the simulation didn't reach either stop state, its tag is "None", and it is lumped with failures.
    reverse = [i for i in dataset if i.tag == "FAILURE" or i.tag == Literals.no_initial_moves or i.tag == Literals.time_out]  
    reverse_array = np.zeros(len(reverse))
    reverse_array[:] = [i.time for i in reverse]

    collision_array = np.zeros(len(dataset))
    collision_array[:] = [i.collision_rate for i in dataset]
    
    return (forward_array, reverse_array, collision_array)

def load_dataset(toehold_length, rate_method):
    # Looks in the standard directory for all matching filenames,
    # loads each file and condenses the results.
    # If there are N files, the result_list will have N non-None items.
    # Each item is a tuple of arrays: (completion times for all successful trials, completion times for all failed trials, initial collition rates for all trials)
    # Thus these three arrays have lengths (S, F, S+F).
    
    dirname = "Data_toehold_{0}".format(toehold_length)
    if not os.path.isdir(dirname):
        print("Error: directory {0} does not exist.\n".format(dirname))
        return None

    files = os.listdir(dirname)
    sample_files = [i for i in files if i.startswith('DATA_' + rate_method)]
    sample_files.sort()
    
    def number_from_fname(filename):
        return filename.rstrip('.dat').split('_')[-1]

    result_list = [None] * (int(number_from_fname(sample_files[-1])) + 1)
    for f in sample_files:
        print("Processing file: {0}".format(f))
        result_list[ int(number_from_fname(f)) ] = process_dataset(load_file(dirname, f))

    return result_list


In [67]:
toehold_length = 7
rate_method = 'Metropolis'
load_dataset(toehold_length, rate_method)

Processing file: DATA_Metropolis_7_0399.dat
Processing file: DATA_Metropolis_7_0400.dat
Processing file: DATA_Metropolis_7_0401.dat


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,